# (1) use sed -r '/^\s*$/d' sv.txt > nonempty_sv.txt

# (2) use split4parts to split nonempty_sv.txt to 4 parts for processing due to PC host mem constraint

# (3) run thru the following SVfilter.py 4 times to process the data to the cleaned folder 
#### example syntax 
#### python SVfilter.py --input_dir SV_CC100_part_aaad --output_dir ./cleaned/SV_CC100_Part4.txt --num_cpu 8

In [1]:
%%writefile SVfilter.py
import sys
import multiprocessing
import os
from multiprocessing import Pool,cpu_count
import nltk
import nltk
from nltk.tokenize import sent_tokenize

import argparse

def cut(sentence):
    sents=sent_tokenizer(line)
    sents=[s for s in sents if len(s)>1]
    if len(sents)>2: # we filter only more than 2 sentences in each doc
        return ''.join(sents)
    else:
        return ''

def main(args):
    num_cpus_to_use=int(args.num_cpu)
    lines=[]
    with open(args.input_dir, 'r', errors='ignore') as f:
        for k, line in enumerate(f, 1):
            line=line.replace('~','')
            line=line.replace('…','')
            line=line.strip()
            if not line:
                continue
            else:
                if len(line)>0 and line not in ['\n','','\r\n','\t',' ']:
                    lines.append(line)
                    k+=1
    print("Finish reading the file with number of non_empty lines ", str(k))
    # 创建进程池
    pool = Pool(num_cpus_to_use)
    data = pool.imap(cut,lines, 512)
    i=0
    outfile = open(args.output_dir, 'a')
    for i,dat in enumerate(data):
        if len(dat) >0:
            #print(i,dat,type(dat))
            outfile.write(dat+'\n')
        if i%1000000==0:
            print("processed {} million lines ...\n".format(str(i/1000000)))
            print("sample dat at i",i, dat)
    print("processing finished !")
    f.close()
    outfile.close()
    
if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('--input_dir', default=None, type=str,  help='input file path')
    parser.add_argument('--output_dir', default=None, type=str, 
                        help='output file path')
    parser.add_argument('--num_cpu', default=48, type=int, 
                        help='number of cpus used for multiprocesing')
    args = parser.parse_args()
    main(args)






Overwriting SVfilter.py


In [22]:
text="Efter en intensiv helg med ridlektionsjobb och sånguppträdande, mycket intryck och människor tar jag en lugn dag hemma idag.\n Blir ju alltid lite dagen-efter efter sånt här, huvudvärk, kan typ sova ett dygn, yr och muskelvärk.Helgen har trots sin intensitet varit väldigt väldigt fin.\nFått umgås med familjen och min älskade sambo.En fin vistelse på hotell osv.Milla har fått vara hos våra vänner Malin och Theo och levt livet - där bor nämligen också hennes bästishund Twix.\nMin hjärna blir ju extremt trött av intryck så tycker lite synd om stackars Martin när det blir såna här intensiva helger.Jag blir liksom helt slut efteråt mentalt och fysiskt.Som ett skal,\n och grubblig och ledsen.Hjärnan kokar över.Men känns skönt att det blir bättre och bättre hela tiden."
i=0
sents=splitter(text)

for sent in sents:
    print("sentence {} : {} \n".format(str(i), sent))
    i+=1

sentence 0 : Efter en intensiv helg med ridlektionsjobb och sånguppträdande, mycket intryck och människor tar jag en lugn dag hemma idag. 

sentence 1 : Blir ju alltid lite dagen-efter efter sånt här, huvudvärk, kan typ sova ett dygn, yr och muskelvärk.Helgen har trots sin intensitet varit väldigt väldigt fin. 

sentence 2 : Fått umgås med familjen och min älskade sambo.En fin vistelse på hotell osv.Milla har fått vara hos våra vänner Malin och Theo och levt livet - där bor nämligen också hennes bästishund Twix. 

sentence 3 : Min hjärna blir ju extremt trött av intryck så tycker lite synd om stackars Martin när det blir såna här intensiva helger.Jag blir liksom helt slut efteråt mentalt och fysiskt.Som ett skal,
 och grubblig och ledsen.Hjärnan kokar över.Men känns skönt att det blir bättre och bättre hela tiden. 



# (4) run the train tokenizer script for the following pretrained tokenizer
### BPE with vocab size 16k/32k and 48k, 
#### note the 16k BPE does not do a good job, seperting the tokenize into individual characters ... 
### WordPiece with vocab size 16k/32k/48k 


# (5) run preprocessing script for both GPT ( original ) and Bert ( preprocessSVdata.py ) and use 32k to start with 
### do NOT use the multiprocessing, it will blow up the system mem and kill job or has broken pipe error 

In [24]:
from megatron.tokenizer.tokenizer import build_debug
vocab_f='/workspace/SVdata/bpe/32k/vocab.json'
merge_f='/workspace/SVdata/bpe/32k/merges.txt'
sv_tokenizer=build_debug(vocab_f,merge_f, tokenizer_type='bpe')


ModuleNotFoundError: No module named 'megatron'